In [1]:
# imports

import os
import re
from dotenv import load_dotenv
from huggingface_hub import login
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import chromadb
import modal

### Internal Classes
from prototypes.testing import Tester
from agents.specialist_agent import SpecialistAgent
from agents.frontier_agent import FrontierAgent

/Volumes/VTG/Dev/C_5/Projects/week8/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [17]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [18]:
openai = OpenAI()

In [ ]:
with open("test.pkl", "rb") as f:
    test = pickle.load(f)

In [ ]:
DB = "products_vectorstore/"

client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection("product")

In [21]:
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
test[0].test_prompt()

In [22]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [23]:
def vector(item): 
    return embedding_model.encode([description(item)])

In [24]:
def find_similars(item): 
    results = collection.query(query_embeddings=vector(item))
    similar_docs = results["documents"][0][:]
    prices = [meta["price"] for meta in results["metadatas"][0][:]]
    
    return similar_docs, prices 

In [ ]:
find_similars(test[1])

In [25]:
def make_context(similars, prices): 
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices): 
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    
    return message

In [26]:
def messages_for(item, similars, prices): 
    system_message = "You estimate prices of items. Reply only with the price, no explanation."
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"  
    user_prompt += item.test_prompt().replace("to the nearest dollar", "").replace("Price is $", "")
    
    return [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": user_prompt}, 
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
docs, prices = find_similars(test[2])

In [ ]:
print(test[2])

In [ ]:
print(make_context(docs, prices))

In [ ]:
print(messages_for(test[5], docs, prices)[1]["content"])

In [27]:
def get_price(s): 
    s = s.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*.\d+|\d+", s)
    return float(match.group()) if match else 0

In [28]:
def gpt_5_mini_rag(item): 
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model="gpt-5-mini", 
        messages=messages_for(item, documents, prices), 
        seed=42
    )
    reply = response.choices[0].message.content 
    return get_price(reply)

In [22]:
test[500].price

139.37

In [23]:
gpt_5_mini_rag(test[500]) 

239.99

## Evaluate the GPT 5-mini with RAG system

In [ ]:
Tester.test(gpt_5_mini_rag, test[:250])

## Ensemble Agent 

This Ensemble Agent combines the result of each different model of the 2 (GPT 5 mini with RAG + Fine tuned Llama 3.1 8B model) that are normalized in some certain ratio.

In [30]:
Pricer = modal.Cls.from_name("pricer-service", "Pricer")

In [33]:
speicalist_agent = SpecialistAgent()

In [36]:
def emsamble(item): 
    ### Price gotten from the GPT model
    price1 = gpt_5_mini_rag(item)
    ### Price gotten from the remote function (from the fine tuned llama model)
    price2 = speicalist_agent.price(description(item))

    ### combine the 2 results -- 80/20
    return price1 * 0.8 + price2 * 0.2

In [ ]:
Tester.test(emsamble, test[:250])

In [35]:
test[:2]

[<Medicom Batman Hush Black Suit Version Batman Real Hero Action Figure = $249.99>,
 <ALTKOL S Hooks for Hanging, 15-Pack S Shaped Hook Heavy Duty Hanging Hooks for Pots, Pans, Plants, Bags, Cups, Clothes, 2.4 Inch Metal (Black) = $3.99>]

In [39]:
agent = FrontierAgent(collection)

In [45]:
agent.price("Quadcast HyperX condenser mic for high quality audio for podcasting")

149.99